In [1]:
import cv2
import numpy as np

In [5]:
cap = cv2.VideoCapture('redball.mp4')
term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS,10, 1)
fgbg=cv2.createBackgroundSubtractorKNN(history=500, dist2Threshold=400.0, 
                                      detectShadows=False)
tracking_start = False

#칼만필터셋업
q = 1e-5 #pnc
r = 0.01 #mnc
dt = 1
KF = cv2.KalmanFilter(4,2,0)
KF.transitionMatrix = np.array([[1,0,dt,0],
                               [0,1,0,dt],
                               [0,0,1,0],
                               [0,0,0,1]],np.float32) #A
KF.measurementMatrix = np.array([[1,0,0,0],
                                [0,1,0,0]],np.float32) #H

In [6]:
while True :
    ret, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lowb = np.array([0,100,100])
    highb = np.array([10,255,255])
    
    #마스크생성
    mask = cv2.inRange(hsv,lowb,highb)
    
    #배경추출
    gaus = cv2.GaussianBlur(mask,(5,5),0)
    image = fgbg.apply(gaus)
    
    #바운딩박스그리기
    nlables, labels, stats, centroids = cv2.connectedComponentsWithStats(image)
    for index, centroid in enumerate(centroids):
        if stats[index][0]==0 and stats[index][1]==0:
            continue
        if np.any(np.isnan(centroid)):
            continue
            
        x,y,width,height,area = stats[index]
        
        if area > 100:
            x,y,w,h,area = stats[index]
            
            #칼만필터 시작
            KF.processNoiseCov = q * np.eye(4,dtype=np.float32) #Q
            KF.measurementNoiseCov = r * np.eye(2,dtype=np.float32) #R
            KF.errorCovPost = np.eye(4,dtype=np.float32) #P0 = I
            
            KF.statePost = np.array([[x],[y],[0.],[0.]],dtype=np.float32)
            
            tracking_start = True
            
            if tracking_start :
                predict = KF.predict()
                
                #칼만정정
                z = np.array([[x],[y]],dtype=np.float32) #measurement
                estimate = KF.correct(z)
                estimate = np.int0(estimate)
                
                #바운딩박스
                x2,y2 = estimate[0][0],estimate[1][0]
                cv2.rectangle(frame,(x2,y2),(x2+w,y2+h),(255,0,0),2)
                
    
    cv2.imshow('tracking',frame)
    if cv2.waitKey(20) == 27:
        break
cv2.destroyAllWindows()